In [6]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys
import os
from tqdm.notebook import tqdm
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)
sr = 16000
dropout = 0.3
half = False
root = '../'

In [3]:
device = 2
torch.cuda.set_device(device)


In [4]:
class OverlayDataSet(torch.utils.data.Dataset):
    def __init__(self, csv, compute_original = False):
        super().__init__()
        self.overlays = pd.read_csv(csv)
        self.speakers = list(set(self.overlays['first_speaker']))
        self.speakers.sort()
        self.spkr2idx = {spkr:i for i, spkr in enumerate(self.speakers)}
        self.compute_original = compute_original
    def __len__(self):
        return len(self.overlays)
    def __getitem__(self, idx):
        overlay = self.overlays.iloc[idx]
        first_segment = np.load(root+overlay['first_file'])/(2**15)
        second_segment = np.load(root+overlay['second_file'])/(2**15)
        third_segment = np.load(root+overlay['third_file'])/(2**15)
        max_len = max(len(first_segment), len(second_segment), len(third_segment))
        #padding to compensate rounding errors
        if max_len>len(first_segment):
            padding = np.zeros(max_len-len(first_segment))
            first_segment = np.concatenate((first_segment, padding))
        
        if max_len>len(second_segment):
            padding = np.zeros(max_len-len(second_segment))
            second_segment = np.concatenate((second_segment, padding))
            
        if max_len>len(third_segment):
            padding = np.zeros(max_len-len(third_segment))
            third_segment = np.concatenate((third_segment, padding))
        
        first_idx  = self.spkr2idx[overlay['first_speaker']]
        second_idx = self.spkr2idx[overlay['second_speaker']]
        third_idx = self.spkr2idx[overlay['third_speaker']]

        target = np.zeros(len(self.speakers))
        target[first_idx] = 1.0
        target[second_idx] = 1.0
        target[third_idx] = 1.0
        
        if self.compute_original:
            return self.make_spectrogram(first_segment), self.make_spectrogram(second_segment),\
                self.make_spectrogram(third_segment), self.make_spectrogram(first_segment+second_segment), target
        else:
            return self.make_spectrogram(first_segment+second_segment+third_segment), target
    def make_spectrogram(self, segment):
        segment = segment[50:-50] # make size 200
        S = librosa.feature.mfcc(segment, sr=16000, n_mfcc=20, dct_type=2, n_fft = 1024, hop_length = 160)[1:14].T
        # 200*13
        S1 = np.diff(S)
        S2 = np.diff(S1)
        S = np.concatenate((S, S1, S2), axis = -1)
        return S

trainset = OverlayDataSet('overlay-train.csv', False)
valset = OverlayDataSet('overlay-val.csv', False)
testset = OverlayDataSet('overlay-test.csv', False)

In [7]:
spec3, target = trainset[0]
plt.figure(figsize = (20, 6))
if trainset.compute_original:
    plt.subplot(131)
    plt.imshow(spec1[0].T)
    plt.subplot(132)
    plt.imshow(spec2[0].T)
    plt.subplot(133)
    plt.imshow(spec3[0].T)
print(spec3.max(), spec3.min(), spec3.shape, target)

265.79355 -200.36958 (200, 36) [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]


<Figure size 1440x432 with 0 Axes>

## Maybe try drastically increasing channel number in residual attention stage to see if it overfits

In [8]:
class Baseline(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn = nn.LayerNorm(36)
        self.reshape =  Lambda(lambda x: x.permute((1, 0, 2))) # L * batch_size * (num_heads*128)
        self.lstm = nn.LSTM(36, 32, 2, batch_first = False, bidirectional = True, dropout = dropout) # L * batch_size * 200 * n_hidden
        self.fc1 = nn.Linear(64, 32)
        self.average = Lambda(lambda x: x.mean(dim = 0)) # batch * n_hidden
        self.tanh = nn.Tanh()
        #self.norm = Lambda(lambda x: torch.nn.functional.normalize(x, p = 2, dim = 1)) # L2 normalize across n_hidden
        self.fc2 = nn.Linear(32, 20)
        self.sigmoid = nn.Sigmoid()
    def forward(self, X):
        X = self.bn(X)
        X = self.reshape(X)
        X, _ = self.lstm(X)
        X = self.fc1(X)
        X = self.average(X)
        X = self.tanh(X)
        X = self.fc2(X)
        X = self.sigmoid(X)
        return X
    
    
overnet = Baseline().cuda(device)
    
# tune hidden layers smaller if overfit
optimizer = torch.optim.Adam(overnet.parameters(), 0.001)

if os.path.exists('models/lstm.pth'):
    print('load model')
    checkpoint = torch.load('models/lstm.pth')
    overnet.load_state_dict(checkpoint['model_state_dict'])
    try:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    except:
        print('cannot load optimizer')
    loss = checkpoint['loss']
    if 'bestacc' in checkpoint:
        bestacc = checkpoint['bestacc']
    else:
        bestacc = 0.0
else:
    print('initializing new model')
    bestacc = 0.0
    
if half:
    overnet.half()  # convert to half precision
    for layer in overnet.modules():
        if isinstance(layer, nn.BatchNorm2d):
            layer.float()
            
overnet.train()
'bestacc:', bestacc

initializing new model


('bestacc:', 0.0)

## Also Do metrics on hitting a single person right

In [11]:
def find_max3(tensor):
    array = tensor.cpu().detach().numpy()
    max3 = []
    for row in array:
        max3.append(np.argsort(row)[::-1][:3])
    return np.array(max3)

def compute_corrects(tensor1, tensor2):
    max_1, max_2 = find_max3(tensor1), find_max3(tensor2)
    batch_size = max_1.shape[0]
    batch_corrects = 0
    for i in range(batch_size):
        if Counter(max_1[i])==Counter(max_2[i]):
            batch_corrects+=1
    return batch_corrects

batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True, pin_memory = True, num_workers = 16)
criterion = torch.nn.BCELoss()

for epoch in range(64):
    running_loss = 0.0
    running_accuracy = 0.0
    for batch_idx, (spec, target) in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()
        spec, target = spec.float(), target.float()
        if half:
            spec, target = spec.half(),target.half()
        spec = spec.cuda(device)
        target = target.cuda(device)

        out = overnet(spec)
        loss = criterion(out, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(overnet.parameters(), 0.5)
        optimizer.step()
                
        running_loss += loss.item()
        running_accuracy += compute_corrects(out, target)/batch_size
        if batch_idx % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f' % 
                  (epoch + 1, batch_idx + 1, running_loss / 200, running_accuracy / 200))
            running_loss = 0.0
            running_accuracy = 0.0
            torch.save({
            'model_state_dict': overnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'models/lstm.pth')

        
        
    corrects = 0
    for batch_idx, (spec, target) in enumerate(tqdm(valloader)):
        spec, target = spec.float(), target.float()
        if half:
            spec, target = spec.half(), target.half()
        spec = spec.cuda(device)
        target = target.cuda(device)
        overnet.eval()
        out = overnet(spec) 
        corrects += compute_corrects(out, target)
    print('val acc:', corrects/len(valset))
    if corrects/len(valset) > bestacc:
        bestacc = corrects/len(valset)
        torch.save({
        'model_state_dict': overnet.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'bestacc': bestacc
        }, 'models/best-lstm.pth')
    overnet.train()
    pass

[1,   200] loss: 0.153 accuracy: 0.438
[1,   400] loss: 0.154 accuracy: 0.431
[1,   600] loss: 0.153 accuracy: 0.435
[1,   800] loss: 0.153 accuracy: 0.438
[1,  1000] loss: 0.153 accuracy: 0.436
[1,  1200] loss: 0.154 accuracy: 0.430
[1,  1400] loss: 0.154 accuracy: 0.441
[1,  1600] loss: 0.153 accuracy: 0.432
[1,  1800] loss: 0.154 accuracy: 0.428
[1,  2000] loss: 0.152 accuracy: 0.440
[1,  2200] loss: 0.153 accuracy: 0.439
[1,  2400] loss: 0.154 accuracy: 0.435
[1,  2600] loss: 0.154 accuracy: 0.434
[1,  2800] loss: 0.155 accuracy: 0.432
[1,  3000] loss: 0.153 accuracy: 0.436
[1,  3200] loss: 0.155 accuracy: 0.437
[1,  3400] loss: 0.153 accuracy: 0.440
[1,  3600] loss: 0.155 accuracy: 0.435
[1,  3800] loss: 0.152 accuracy: 0.443
[1,  4000] loss: 0.154 accuracy: 0.436
[1,  4200] loss: 0.154 accuracy: 0.432
[1,  4400] loss: 0.154 accuracy: 0.434
[1,  4600] loss: 0.153 accuracy: 0.442
[1,  4800] loss: 0.154 accuracy: 0.426
[1,  5000] loss: 0.153 accuracy: 0.437
[1,  5200] loss: 0.153 ac


val acc: 0.4497227263561202


[2,   200] loss: 0.151 accuracy: 0.445
[2,   400] loss: 0.152 accuracy: 0.436
[2,   600] loss: 0.153 accuracy: 0.442
[2,   800] loss: 0.152 accuracy: 0.441
[2,  1000] loss: 0.152 accuracy: 0.441
[2,  1200] loss: 0.153 accuracy: 0.439
[2,  1400] loss: 0.153 accuracy: 0.436
[2,  1600] loss: 0.154 accuracy: 0.437
[2,  1800] loss: 0.153 accuracy: 0.436
[2,  2000] loss: 0.152 accuracy: 0.439
[2,  2200] loss: 0.153 accuracy: 0.435
[2,  2400] loss: 0.153 accuracy: 0.438
[2,  2600] loss: 0.152 accuracy: 0.444
[2,  2800] loss: 0.153 accuracy: 0.436
[2,  3000] loss: 0.152 accuracy: 0.436
[2,  3200] loss: 0.153 accuracy: 0.437
[2,  3400] loss: 0.152 accuracy: 0.440
[2,  3600] loss: 0.153 accuracy: 0.438
[2,  3800] loss: 0.154 accuracy: 0.431
[2,  4000] loss: 0.153 accuracy: 0.433
[2,  4200] loss: 0.154 accuracy: 0.438
[2,  4400] loss: 0.153 accuracy: 0.437
[2,  4600] loss: 0.153 accuracy: 0.432
[2,  4800] loss: 0.153 accuracy: 0.441
[2,  5000] loss: 0.154 accuracy: 0.438
[2,  5200] loss: 0.154 ac


val acc: 0.45617059891107076


[3,   200] loss: 0.151 accuracy: 0.440
[3,   400] loss: 0.152 accuracy: 0.439
[3,   600] loss: 0.151 accuracy: 0.442
[3,   800] loss: 0.152 accuracy: 0.440
[3,  1000] loss: 0.151 accuracy: 0.442
[3,  1200] loss: 0.153 accuracy: 0.436
[3,  1400] loss: 0.151 accuracy: 0.443
[3,  1600] loss: 0.152 accuracy: 0.438
[3,  1800] loss: 0.152 accuracy: 0.444
[3,  2000] loss: 0.151 accuracy: 0.447
[3,  2200] loss: 0.153 accuracy: 0.436
[3,  2400] loss: 0.151 accuracy: 0.441
[3,  2600] loss: 0.153 accuracy: 0.440
[3,  2800] loss: 0.151 accuracy: 0.446
[3,  3000] loss: 0.151 accuracy: 0.445
[3,  3200] loss: 0.152 accuracy: 0.440
[3,  3400] loss: 0.151 accuracy: 0.440
[3,  3600] loss: 0.152 accuracy: 0.439
[3,  3800] loss: 0.153 accuracy: 0.439
[3,  4000] loss: 0.151 accuracy: 0.443
[3,  4200] loss: 0.152 accuracy: 0.442
[3,  4400] loss: 0.153 accuracy: 0.435
[3,  4600] loss: 0.152 accuracy: 0.444
[3,  4800] loss: 0.153 accuracy: 0.440
[3,  5000] loss: 0.153 accuracy: 0.441
[3,  5200] loss: 0.151 ac

...


val acc: 0.48649425287356324


[32,   200] loss: 0.142 accuracy: 0.479
[32,   400] loss: 0.142 accuracy: 0.473
[32,   600] loss: 0.142 accuracy: 0.476
[32,   800] loss: 0.141 accuracy: 0.473
[32,  1000] loss: 0.142 accuracy: 0.474
[32,  1200] loss: 0.143 accuracy: 0.466
[32,  1400] loss: 0.138 accuracy: 0.485
[32,  1600] loss: 0.142 accuracy: 0.471
[32,  1800] loss: 0.142 accuracy: 0.473
[32,  2000] loss: 0.141 accuracy: 0.485
[32,  2200] loss: 0.142 accuracy: 0.472
[32,  2400] loss: 0.141 accuracy: 0.476
[32,  2600] loss: 0.141 accuracy: 0.482
[32,  2800] loss: 0.142 accuracy: 0.474
[32,  3000] loss: 0.143 accuracy: 0.465
[32,  3200] loss: 0.141 accuracy: 0.478
[32,  3400] loss: 0.142 accuracy: 0.473
[32,  3600] loss: 0.141 accuracy: 0.476
[32,  3800] loss: 0.143 accuracy: 0.469
[32,  4000] loss: 0.143 accuracy: 0.470
[32,  4200] loss: 0.144 accuracy: 0.467
[32,  4400] loss: 0.143 accuracy: 0.473
[32,  4600] loss: 0.142 accuracy: 0.475
[32,  4800] loss: 0.142 accuracy: 0.470
[32,  5000] loss: 0.143 accuracy: 0.467



val acc: 0.48944847751562814


[33,   200] loss: 0.141 accuracy: 0.475
[33,   400] loss: 0.142 accuracy: 0.477
[33,   600] loss: 0.140 accuracy: 0.482
[33,   800] loss: 0.142 accuracy: 0.466
[33,  1000] loss: 0.141 accuracy: 0.477
[33,  1200] loss: 0.141 accuracy: 0.475
[33,  1400] loss: 0.143 accuracy: 0.473
[33,  1600] loss: 0.142 accuracy: 0.481
[33,  1800] loss: 0.141 accuracy: 0.478
[33,  2000] loss: 0.142 accuracy: 0.478
[33,  2200] loss: 0.141 accuracy: 0.475
[33,  2400] loss: 0.144 accuracy: 0.464
[33,  2600] loss: 0.141 accuracy: 0.473
[33,  2800] loss: 0.141 accuracy: 0.475
[33,  3000] loss: 0.143 accuracy: 0.472
[33,  3200] loss: 0.141 accuracy: 0.479
[33,  3400] loss: 0.143 accuracy: 0.470
[33,  3600] loss: 0.144 accuracy: 0.468
[33,  3800] loss: 0.142 accuracy: 0.468
[33,  4000] loss: 0.144 accuracy: 0.471
[33,  4200] loss: 0.143 accuracy: 0.469
[33,  4400] loss: 0.142 accuracy: 0.471
[33,  4600] loss: 0.140 accuracy: 0.481
[33,  4800] loss: 0.141 accuracy: 0.477
[33,  5000] loss: 0.142 accuracy: 0.477


KeyboardInterrupt: 

In [12]:
checkpoint = torch.load('models/best-lstm.pth')
overnet.load_state_dict(checkpoint['model_state_dict'])

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, pin_memory = True, num_workers = 16)
        
corrects = 0
for batch_idx, (spec, target) in enumerate(tqdm(testloader)):
    spec, target = spec.float(), target.float()
    if half:
        spec, target = spec.half(), target.half()
    spec = spec.cuda(device)
    target = target.cuda(device)
    overnet.eval()
    out = overnet(spec) 
    corrects += compute_corrects(out, target)
print('test acc:', corrects/len(testset))
overnet.train()
pass


test acc: 0.49054748941318815
